In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2 as cv
from opencv_hacks import *


plt.rcParams["figure.figsize"] = (40,20)

%matplotlib inline

# **Multiple View Geometry in Computer Vision**

This is my lecture notes taken while watching Daniel Cremer's *Multiple View Geometry in Computer Vision* course on Youtube.

## **Chapter 0: Introduction**


In computer vision, we would like to obtain a 3D model from 2D images. Some applications of 3D modelling including face recognition (recognizing a 3D face is easier), computer graphics (relighting a scene becomes easier), plastic surgery etc. 3D reconstruction can be thought of as infinite dimensional optimization of reshaping a sphere to a desired shape. The issue is that the energy functions being optimized are generally non-convex.

Even though images will be discrete, there is value to treating an image as a function $f : \Omega \to \mathbb{R}^3$ where $\Omega \subseteq \mathbb{R}^{3}$. This will allow for algorithms that are independent of the choice of the grid and allow variational techniques.


A classical approach to image de-noising is $$u_{\text{den}} = \arg \min_{u} \int_{\Omega} (u - f)^2 dx + \lambda \int_{\Omega} \lvert \nabla u \rvert dx$$ Here $u$ is an unspecified function, and the second term above is the *regularization* term which ensures that the image is de-noised (function is roughly smooth). 

## **Chapter 1: Mathematical Background**

We deal with 3D reconstruction using methods of linear algebra. 

Given two matrices $A \in \mathbb{R}^{m \times n}$ and $B \in \mathbb{R}^{k \times \ell}$ we have $A \otimes B = [a_{(i+1),(j+1)}b_{(r,s)}]_{(k \cdot i + r, \ell \cdot j + s)} = \begin{bmatrix}a_{11} B & \ldots & a_{1n} B \\ \vdots &\ddots & \vdots \\ a_{m1}B & \ldots & a_{mn}B \end{bmatrix} \in \mathbb{R }^{mk \times nl}$ where $r,s \in [k] \times [\ell]$.   

We have $u^T A v = (v \otimes u)^T A^s$ where $A = [a_1 \mid \ldots \mid a_n]$ and $A^s = \begin{bmatrix} a_1 \\ \vdots \\ a_n \end{bmatrix} \in \mathbb{R}^{m \times n}$. Indeed, we simply have $v \otimes u = uv^T$

A group $G$ has a matrix representation if there exists an injective embedding $R : \verb|G| \to \verb|GL|(n)$. This allows analysis of more abstract groups by looking at properties of the respective matrix groups. 

The affine transformation group can be represented by the matrix representation $\begin{bmatrix}  A & \mathbf{b} \\ \mathbf{0}^T & 1 \end{bmatrix}$. The Euclidean group can be similarly represented as $\begin{bmatrix} R & \mathbf{t} \\ \mathbf{0}^T & 1 \end{bmatrix}$ where $\mathbf{R} \in O(n)$. There is an associated Special Euclidean group $SE(n)$ where the matrices $R$ are restricted to being in $SO(n)$. These are the group of motions of a camera.

Let $A \in \mathbb{R}^{m \times n}$ and $B \in \mathbb{R}^{n \times k}$. Then $\text{rank}(A) + \text{rank}(B) - n \leq \text{rank}(AB) \leq \min \{\text{rank}(A), \text{rank}(B)\}$. 

### **Linear Algebra (contd.)**



We will denote the spectrum of a matrix $A$ by $\sigma(A)$. This is the set of eigenvalues of $A$.

#### **The Spectral Theorem**

 If $A$ is symmetric, then $A$ has real eigenvalues. This can be shown by the following argument:

* If $A$ is self-adjoint ($A^* = A$) then for any eigenvalue $\lambda$ of $A$ we have $Av \cdot v = \lambda \lvert v \rvert^2 = v \cdot Av = \overline{\lambda} \cdot\lvert v \rvert^2$. Thus $\lambda$ is real.
* If $A$ is self-adjoint, then $A$ is diagonalizable. Let $Ax = \lambda x$ where $\lambda$ is real. Write $V = E_{\lambda} \oplus E_{\lambda}^{\perp}$. Now $A \mid_{E_{\lambda}^{\perp}}$ is defined. Indeed, if $z \in E_{\lambda}^\perp$, we have $z \cdot v = 0$ for $v \in E_{\lambda}$. Now $Az \cdot v = z \cdot Av = \lambda z \cdot v = 0$, so $Az \in E_{\lambda}^\perp$ as well. This means, $A$ itself splits over the decomposition. Now the restriction of $A$ is also self-adjoint, since the inner product is just the restriction of the inner product on the larger space, so we are done by induction.
* The above process shows that $V$ has an orthogonal basis of eigenvectors for $A$. 
* Now if $A$ is symmetric, we can choose eigenvectors for $A$ that are real. Indeed, $A$ has a real eigenvector because $A(x + iy) = 0$ implies $Ax = 0 = Ay$ and we may just repeat the above proof for the real case.

If $A \succeq 0$ and $\lambda_1 \geq \ldots \geq \lambda_n \geq 0$ are the eigenvalues of $A$, then $\max_{\lvert x \rvert = 1} x^TAx = \lambda_1$ and $\min_{\lvert x \rvert = 1} x^T A x = \lambda_n$. Indeed, write $A = U^T \Sigma U$ so that $x^T A x = (Ux)^T \Sigma (Ux)$. Since $U$ is orthogonal, this reduces to computing the extrema of $y^T \Sigma y$ where $\lvert y\rvert = 1$. But $y^T \Sigma y = \sum_{j=1}^{n} \lambda_j y_j^2$ which can easily be seen to be between $\lambda_n$ and $\lambda_1$.

Some norms on matrices include $\lvert\lvert A \rvert\rvert_a = \max_{\lvert x_2 \rvert_2 = 1} \lvert Ax \rvert_2 = \max_{\lvert x \rvert_2 = 1}\sqrt{x^T (A^TA) x}$ and $\lvert\lvert A \rvert\rvert_f = \sqrt{\verb|trace|(A^TA)}$. Since $A^TA$ is positive semi-definite, we can write $A^TA = V\text{diag}(\sigma_1^2,\ldots, \sigma_n^2)V^T$ where $\sigma_1 \geq \ldots\geq \sigma_n$ are the singular values of $A$. Then $\lvert\lvert A \rvert\rvert_2 = \sigma_1$ and $\lvert\lvert A \rvert\rvert_f = \sqrt{\sigma_1^2 + \ldots + \sigma_n^2}$.

The spectral theorem states that if $A$ is a normal matrix ($AA^* = A^*A$), then $A$ is diagonalizable by a unitary matrix. We will prove this. 

* Let $A$ be a complex matrix. Then there is an orthonormal basis such that $A$ is upper triangular. Pick an eigenvalue $\lambda$ for $A$ (this exists because $\det(A - \lambda I) = 0$ must have a complex root). Decompose the space as $V = E_\lambda \oplus E_\lambda^\perp$. Pick an orthonormal basis $v_1,\ldots, v_n, w_1,\ldots, w_m$ corresponding to this decomposition. Then the matrix of $A$ with respect to this basis will have the form $\begin{bmatrix} \lambda I & B \\ 0 & C \end{bmatrix}$. That is, $Aw_j = BV + CW$. Applying the orthogonal projection onto $E_\lambda^\perp$, we get $PAw_j = Cw_j$. Thus $C : E_\lambda^\perp \to E_\lambda^\perp$ is a transformation and by induction, is orthogonally upper-triangularizable. This completes the proof.
* Write $A = UBU^*$ where $B$ is upper triangular. Then $A^*A = UB^*BU^*$ and $AA^* = UB^*BU^*$. Thus, $AA^* = A^*A$. This means that $B$ is a normal upper triangular matrix. 
* Any normal upper triangular matrix is diagonal. Now $(A^*A)_{ij}$ is $\langle v_i,v_j\rangle$, where $v_i$ is the $i$ th column of $A$.  $AA^* = \langle w_i, w_j\rangle$ where $w_i$ is the $i$ th row of $A$. So $\lvert v_i \rvert^2 = \lvert w_i\rvert^2$. First we have $\lvert a_{11}\rvert^2 = \lvert a_{11}\rvert^2 + \ldots + \lvert a_{1n}\rvert^2$. This means the first row is zero except possibly at the diagonal. Then for the second row and column we have a similar relation, which forces the only non-zero entry on the second entry to be on the diagonal. This concludes the proof.
* Thus, any normal matrix is unitarily diagonalizable. 

#### **Skew-Symmetric Matrices**

If $A$ is skew-symmetric, then all the eigenvalues of $A$ are purely imaginary (we treat 0 as both real and purely imaginary). For skew-symmetric matrix, there exists a block-diagonal matrix $\Lambda = \text{diag}(A_1,\ldots, A_m, 0, \ldots, 0)$ with real skew-symmetric matrices of the form $A_i = \begin{bmatrix}0 & a_i \\ -a_i & 0 \end{bmatrix}$. In particular, the rank of any skew-symmetric matrix is even.

In our case, suppose that $A$ is a skew-symmetric matrix. This means $A^*= -A$. Clearly $A$ is normal, so write $A = UDU^*$. For any eigenvalue $\lambda$ and eigenvector $v$ of $A$, we have $\lambda \lvert v\rvert^2 = Av \cdot v = -v \cdot Av = -\overline{\lambda} \lvert v\rvert^2$. Thus, $\overline{\lambda} = -\lambda$, and so $\lambda$ is purely imaginary. 

Now, if $x + iy$ is an eigenvector for a non-zero eigenvalue $i\lambda$ for $A$, we have $A(x + iy) = Ax + iAy = i\lambda(x + iy) = -\lambda y + i \lambda x$. Thus, $Ax = -\lambda y$ and $Ay = \lambda x$. But this means that $A(y + ix) = Ay + iAx = \lambda x - i\lambda y = -\lambda i (y + ix)$. Thus, if $x + iy$ is an eigenvector of $i\lambda$, $y+ix$ is an eigenvector of $-i\lambda$. Note that both $x,y$ have to be non-zero since $Ay = x$ and $Ax = -y$. 

Thus, we can orthogonally diagonalize $A$ using bases that correspond in this manner. If we pick $x + iy$ for $i\lambda$, we can ensure that we pick $y+ix$ for $-i\lambda$. These will still be orthonormal since if $0 =(x + iy) \cdot (u + iv) = (x + iy)(u - iv)= (xu + yv) + i(yu - xv)$, we have $xu + yv = 0$ and $yu - xv = 0$. These relations will continue to hold if we swap $x, u$ with $y,v$.

So we have an orthogonal basis $(x_1 + i y_1), (y_1 + ix_1), \ldots, (x_k + iy_k), (y_k + ix_k), z_{1}, \ldots, z_\ell$, where $z_1,\ldots, z_\ell$ is a real basis for the null space (the dimension of the real null space is equal to the dimension of the complex null space since the real echelon form will also be the complex echelon form). Let us show that $x_1, y_1, x_2, y_2, \ldots, x_k, y_k, z_1, \ldots, z_\ell$ forms an orthogonal basis such that the matrix of $A$ is in the desired form above.

We have $\langle x + iy, y+ix\rangle = 0$, so $x \cdot y = 0$. We now want to compare $x_a$ to $x_b$, $y_a$ to $y_b$, and $x_a$ to $y_b$. We have $(x_a,y_a) \cdot (x_b,y_b) = 0 = (x_a,y_a) \cdot (y_b, x_b) = (y_a,x_a) \cdot (y_b,x_b)$. Let us expand these three relations:

* $(x_a, y_b) \cdot (x_b, y_b) = 0$ means that $x_a \cdot x_b + y_a \cdot y_b = 0$ and $y_a \cdot x_b = x_a \cdot y_b$.
* $(x_a, y_a) \cdot (y_b,x_b) = 0$ means that $x_a \cdot y_b + y_a \cdot x_b = 0$ and $y_a \cdot y_b = x_a \cdot x_b$. 

But together, these show $x_a \cdot x_b = 0 = y_a \cdot y_b = x_a \cdot y_b$. 

Thus, we have found an orthogonal basis under which $A$ has the desired form. Normalizing this basis doesn't affect this block decomposition. If $Ax = cy$ and $Ay = -cx$ where $c \neq 0$, then $c \cdot \lvert y\rvert^2 = Ax \cdot y = x \cdot (-Ay) = x \cdot cx = c \lvert x\rvert^2  $. Thus, $\lvert x \rvert = \lvert y\rvert$, and so we may rescale both $x$ and $y$ to be unit vectors. 

This shows that the rank of a skew-symmetric matrix is always even.

A skew-symmetric matrix that shows up in computer vision is the hat operator $$\hat{u} = \begin{bmatrix} 0 & -u_3 & u_2 \\ u_3 & 0 & -u_1 \\ u_1 & - u_2 & 0 \end{bmatrix}$$ In fact, $\hat{u} \cdot v = u \times v$.

#### **Singular Value Decomposition (SVD)**

Suppose that $A$ is an $m \times n$ matrix. Since $A^*A$ is positive semi-definite, it can be orthogonally diagonalized with eigenvalues $\sigma_1^2 \geq \ldots \geq \sigma_r^2  > 0$ where $r$ is the rank of $A$. Pick an orthonormal basis $v_1,\ldots, v_n$ such that $A^*A(v_j) = \sigma^2 v_j$ for $j = 1,\ldots, r$ and $A^*A(v_j) = 0$ for $j > r$. Let $u_1, \ldots, u_r \in \mathbb{R}^{m}$ be such that $Av_j = \sigma_j u_j$ for $j = 1,\ldots, r$. Then we have $$\langle u_j, u_k \rangle = \frac{1}{\sigma_j\sigma_k}\langle Av_j, Av_k \rangle = \langle v_j, A^*A v_k \rangle = \frac{\sigma_k}{\sigma_j} \langle v_j, v_k\rangle$$ Thus, $u_1, \ldots, u_r$ form an orthonormal set. Extend this to an orthonormal basis $u_1,\ldots, u_m$ of $\mathbb{R}^m$. Then if $U,V$ are matrices with columns $u_i$ and $v_i$ respectively, we have $AV = U\Sigma$, as desired.

Any square matrix $A$ maps the unit sphere into an ellipsoid with semi-axis $\sigma_j u_j$.

The singular value decomposition can be used to construct a generalized (More-Penrose) inverse. We define $A^\dagger = V\Sigma^\dagger U^T$, where $\Sigma^\dagger = \begin{bmatrix} \Sigma_1^{-1} & 0 \\ 0 & 0 \end{bmatrix}$. We have $AA^\dagger A = A$ and $A^\dagger A A^\dagger = A^\dagger$. This can easily be checked from the fact that $A^\dagger(u_j) = \frac{1}{\sigma_j} v_j$ for $j = 1,\ldots, r$ and $A^\dagger(u_j) = 0$ for $j > r$.

In fact, $x = A^\dagger b$ is the element with smallest norm $\lvert x\rvert$ among all minimizers of $\lvert Ax - b\rvert^2$. Indeed, if we work in the basis corresponding to $U$ and $V$, the system $Ax = b$ $\Sigma x = b$ and the minimizer will have predetermined coordinates among the first $r$ entries (precisely $A^\dagger b$). The values for the remaining coordinates can be arbitrary, so the one with minimum norm will have to be $A^\dagger b$.

## **Chapter 2: Representing a Moving Scene**

We will start with the 2 view setting where we have a moving camera photographing a static scene. The goal of 3D reconstruction is to reconstruct the three-dimensional structure of the world from a set of two-dimensional views. It is a classical *ill-posed* problem, so we will need to make additional assumptions. The joint estimation of camera motion and 3D location is called *structure and motion* or *visual SLAM* (Simultaneous Localization and Mapping).

 By identifying tangent spaces at each point of $\mathbb{R}^3$, we can define distances, lengths of curves, areas, and volumes. For example $I(\gamma) = \int_{0}^{1} \lvert \dot{\gamma} \rvert ds$ is the arc length of a curve $\gamma : [0,1] \to \mathbb{R}^3$.

There is a natural linear identification $ \hat{\cdot} : \mathbb{R}^3 \to \mathfrak{so}(3)$. A *rigid body motion* is a family of maps $g_t : \mathbb{R}^3 \to \mathbb{R}^3$ for $t \in [0,1]$ which preserve lengths and cross products. This means that rigid body motions also preserves volumes.

Any isometry $T : \mathbb{R}^3 \to \mathbb{R}^3$ such that $T(0) = 0$ is a linear map. This means that $T$ must preserve inner products as well and hence be orthogonal. Indeed, suppose that $T(0) = 0$ and that $T$ is an isometry, i.e $\lvert T(x) - T(y)\rvert = \lvert x-y\rvert$. This means $\lvert T(x)\rvert =\lvert x\rvert$ and since the identity $$\lvert x-y\rvert^2 = \lvert x\rvert^2 + \lvert y\rvert^2 - 2\langle x,y\rangle $$ holds, we have $$\begin{align*}
-2\langle x,y \rangle &= \lvert x\rvert^2 + \lvert y\rvert^2 - \lvert x-y\rvert^2\\
&= \lvert T(x) \rvert^2 + \lvert T(y)\rvert^2 - \lvert T(x) - T(y)\rvert^2\\
&= -2\langle T(x), T(y) \rangle
\end{align*}$$ Thus, $T$ preserves inner products, and so $T(e_1), T(e_2), T(e_3)$ is an orthonormal basis. Further, $T(x) \cdot T(e_i) = x \cdot e_i$ so that $T(ae_1 + be_2 + ce_3) = aT(e_1) + bT(e_2) + cT(e_3)$. Thus, $T$ is linear.


The condition that the cross product is preserved means orientation is preserved, and our one parameter subgroup lies in $SE(n)$.

Rotation groups are smooth and we can make sense of infinitesimal rotations. If we have a family of rotation matrices $R(t)$, we can easily derive that we must have $\dot{R}R^T + R^T \dot{R} = 0$. Thus, $\dot{R}(t) = \hat{w}(t) R(t)$ where $\hat{w}(t)$ is a skew symmetric matrix. So a skew symmetric matrix gives a first order approximation of a rotation $R(dt)\approx  R(0) + dR = I + \hat{w}(0) dt$. 

Using exponential coordinates for rotation allows for easier optimization. Without this, we would have to perform constrained optimization in the special orthogonal group. 

We can solve the differential equation $R(0) = I$ and $\dot{R} = \hat{w} R$ with $R(t) = \exp(\hat{w} t)$. So we have a map $\exp : \mathfrak{so}(3) \to \text{SO}(3)$. For any rotation matrix $R$ we can find a map $\hat{w}$ such that $R = \exp\hat{w}$. If $R = (r_{ij}) \neq I$, we can represent $w$ by $ \cos \lvert w\rvert = \frac{\verb|Trace|(R) - 1}{2}$ and $\frac{w}{\lvert w\rvert} = \frac{1}{2 \sin \lvert w\rvert} \begin{bmatrix} r_{32} - r_{23} \\ r_{13} - r_{31} \\ r_{21} - r_{12}\end{bmatrix}$. 

In fact, there is a simple formula $\exp \hat{w} = I + \frac{\hat{w}}{\lvert w\rvert} \sin(\lvert w \rvert) + \frac{\hat{w}^2}{\lvert w\rvert^2} ( 1 - \cos \lvert w\rvert)$. This is called *Rodrigue's formula* and is useful in practice. One application is optimizing functions over the (special) orthogonal group. 

We have $SE(3) = \left\{g = \begin{bmatrix} R & \mathbf{t} \\ \mathbf{0}^t & 1 \end{bmatrix} \mid R \in SO(3)\right\}$. Now consider a path $g(t) = \begin{bmatrix} R(t) & \mathbf{t}(t) \\ \mathbf{0}^T & 1 \end{bmatrix}$. Then we have $\dot{g} g^{-1} = \begin{bmatrix}\dot{R}R^T & \dot{\mathbf{t}} - \dot{R}R^T\mathbf{t} \\ \mathbf{0}^T & 0 \end{bmatrix} = \begin{bmatrix} \hat{w}(t) & v(t) \\ 0 & 0 \end{bmatrix} = \hat{\xi}(t)$. This the Lie algebra of *twists* (infinitesimal rotations and translations).

We can construct exponential coordinates for SE(3) as well. This uses the formula $$\exp \hat{\xi} = \begin{bmatrix} \exp \hat{w} & \frac{(I - \exp \hat{w}) \hat{w} v + ww^Tv}{\lvert w\rvert} \\ \mathbf{0}^T & 1\end{bmatrix}$$

When observing a scene from a moving camera, the coordinates and velocity of points in camera coordinates will change. We will use a rigid body transformation $g(t) \in \verb|SE|(3)$ to represent the motion from a fixed world frame to the camera frame at time $t$. So then we will have $X(t) = R(t)X_0 + T(t) = g(t)X_0$ where we use homogeneous coordinates in the second case. 

Given two times $t_1$ and $t_2$ we will denote the transformation from the points in the first frame to the points in the second frame by $g(t_1,t_2)$. This satisfies $g(t_3, t_1) = g(t_3, t_2)g(t_2, t_1)$.



Suppose that we have a point $X_0$ in frame $t$ given by $X(t) = g(t)X_0$. Then the apparent velocity of the point caused by the moving camera is $\dot{X}(t) = \dot{g}(t)X_0 = \dot{g}(t)g(t)^{-1} X(t)$. But we can introduce twist coordinates $\hat{V}(t) = \dot{g}(t)g(t)^{-1} = \begin{bmatrix} \hat{w}(t) & v(t) \\ \mathbf{0} & 0\end{bmatrix} \in \mathfrak{se}(3)$. So we have $\dot{X}(t) = \hat{V}(t)X_0$. This corresponds to the equation $\dot{X}(t) = \hat{w}(t)X(t) + v(t)$ in inhomogeneous coordinates.

Suppose that a viewer in another frame $A$ is displaced relative to the current frame by a transformation $Y(t) = g_{xy} X(t)$. Then $\dot{Y}(t) = g_{xy}\dot{X}(t) = g_{xy} \hat{V}(t) g_{xy}^{-1} Y(t)$. Thus, the relative velocity of points observed from camera frame is represented by the twist $$\hat{V}_y = g_{xy} \hat{V} g_{x,y}^{-1} = \verb|ad|_{g_{xy}}(\hat{V})$$ The map $\verb|ad|$ is the adjoint map on $\mathfrak{se}(3)$.

## **Chapter 3: Perspective Projection** 

The basic perspective image equation is that $x = -\frac{f}{Z}X$ and $y = -\frac{f}{Z}Y$. We will drop the signs by assuming the image plane lies ahead of the center of projection. 

So we have $$Z \begin{bmatrix} x \\ y \\ 1\end{bmatrix} = \underbrace{\begin{bmatrix}f & 0  & 0 \\ 0 & f & 0 \\ 0 &  0& 1 \end{bmatrix}}_{K_f}\underbrace{\begin{bmatrix} 1 & 0 & 0  & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \end{bmatrix}}_{\Pi_0}\begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}$$ We will assume that the depth is a constant $\lambda$ (this means that we are assuming that the camera is sufficiently away from the scene).

Due to rigid motion of the camera, the point $X$ in camera coordinates is given as a function of the point in world coordinates $X_0$ by $X = RX_0 + T$. In homogeneous coordinates, $$X = gX_0 = \begin{bmatrix} R & T \\ \mathbf{0}^T & 1 \end{bmatrix}$$ Thus, $\lambda x = K_f\Pi_0 g X_0$. 

Now to obtain pixel coordinates, we will need to further translate and scale the image. This means that we need to multiply the result above with the matrix $K_s = \begin{bmatrix}s_x & s_\theta & o_x \\ 0 & s_y & o_y \\ 0 & 0 & 1 \end{bmatrix}$. The product $K_sK_f$ is called the *intrinsic parameter* matrix. We have $\lambda x = K\Pi_0 g X_0 = \Pi X_0$ where $\Pi = K \Pi_0 g = (KR, KT)$ is the *general projection matrix*.

Dividing by $\lambda$, we get $x' = \frac{\pi_1^T X_0}{\pi_3^T X_0}$ and $y' = \frac{\pi_2^T X_0}{\pi_3^T X_0}$, where $\pi_1, \pi_2, \pi_3$ are the three rows of $\Pi$. This is just the dehomogenization process.
 

We will sometimes assume that we measure lengths in units of the focal length so that we may assume $f = 1$. But in general, we may not have access to the internal parameters of the camera and this makes the reconstruction problem much more difficult. 

In the intrinsic parameter matrix $K = \begin{bmatrix} fs_x & fs_\theta & o_x \\ 0 & fs_y & o_y \\ 0 & 0 & 1 \end{bmatrix}$ the parameters can be interpreted as follows:

* $o_x,o_y$: the $x,y$ coordinate of the principal point in pixels.
* $fs_x = \alpha_x, fs_y = \alpha_y$: the size of unit length in horizontal and vertical pixels
* $\frac{\alpha_x}{\alpha_y}$: the aspect ratio.
* $fs_\theta$: skew of the pixel, often close to zero.

We can also consider a spherical projection surface given yb the surface $\mathbb{S}^2$. In this case, the projection of a point $X$ is just $x = \frac{X}{\lvert X\rvert}$, and so the pixel coordinates are given by $\lambda x' = K \Pi_0 gX_0$ where $\lambda = \sqrt{X^2 + Y^2 + Z^2}$.

All cameras often have some form of *radial distortion*. This means that straight lines are transformed to curves in the image. An effective model for such distortions is $$\vec{x} = \vec{x}_d \left(1 + a_1 r + a_2 r^2 \right)$$ where $\vec{x}_d = (x_d,y_d)$ is the distorted point and $r^2 = x_d^2 + y_d^2$. If a calibration rig is available, we can estimate the parameters $a_1$ and $a_2$.

A more general is $$\vec{x} = \vec{c} + f(r)(\vec{x}_d - \vec{c})$$ where $f(r) = 1 + a_1 r + a_2 r^2 + a_3 r^3 + a_3 r^4$. Here $r = \lvert \vec{x} - \vec{c}\rvert$ is the distance to an arbitrary center of distortion $\vec{c}$.

Suppose that $X = X_0 + \mu V$ is a line in 3D. Then $x \sim \Pi X_0 + \mu \Pi V$. This means that the preimage of a line is a plane. If we have multiple views, we can find the intersection of these preimage planes to identify the original line in 3D. This is is the plane with the same normal vector as $\ell$ in the image plane.

## **Chapter 4: Estimating Point Correspondence**

In practice we don't observe points or lines, but brightness or color values at individual pixels. In order to transfer from this photometric representation to a geometric representation of the scene, one can identify points with *characteristic image features* and try to associate these points with corresponding points in the other frames. This allows us to throw away a large amount of potentially useful information in each image. Establishing such point correspondences allow us to track the flow of objects over time.

### **Introduction**

This problem is a challenge because viewpoint chan ges can lead to distortions. For example, a non-lambertian surface could have very different appearances locally from different views. In general, objects may also deform in a non-rigid manner. Further, there may be occlusions or missing objects (for example, a pair of glasses would occlu1de the eyes).

In point matching, one distinguishes two cases:

* *Small deformation*. The deformation from one frame to the other is assumed to be (infinitesimally) small. In this case the displacement from one frame to the other can be estimated by classical optical flow estimation like Lucas-Kanade or Horn-Shunck. These methods allow to model dense deformation fields and gives a displacement for each pixel in the image. Typically, one can also track the displacement of a few feature points and this is generally faster. A moving camcorder would be in the small baseline case.
* *Wide baseline stereo*. In this case, the displacement is assumed to be large. A dense matching of all points is in general computationally infeasible so one typically selects a small number of feature points in each image and develops efficient methods to find appropriate pairs of points.

We assume the scene is static and the camera is moving so the transformations of all points is given by $$x_2  = h(x_1) = \frac{1}{\lambda_2(X)} \left(R\lambda_1(X) x_1 + T\right)$$ If $x_1$ is in projective coordinates, $\lambda_1(X)x_1$ is the 3D coordinate of the original point.

We can approximate this transformation by $h(x) = Ax + b$. If we define $u(x) = h(x) - x$ and then we can write $$u(x) = S(x)p = \begin{pmatrix}x & y & 1  & 0 & 0 & 0 \\ 0 & 0 & 0 & x & y & 1 \end{pmatrix} \begin{pmatrix}p_1 & p_2 & p_3 & p_3 & p_4 & p_5 & p_6 \end{pmatrix}^T$$                 

### **Optical Flow Estimation**

The goal of optical flow estimation is to estimate the motion of the *image field*. The Lucas-Kanade method was more popular previously since it was a sparse method, but now the Horn-Shunck method is more popular with increased computational capabilities. 



For optical flow estimation color doesn't seem to bring much advantage, so we may focus only on brightness. The key assumption in both methods is brightness consistency. Suppose that $x(t)$ denotes a moving point at time $t$, and $I(x,t)$ is a video sequence. Then $$I(x(t), t) = \text{const}$$ for all $t$. The brightness constancy assumption need to hold exactly (if a sun comes out, a point will change brightness), but it is a reasonable assumption that it holds approximately.

We may take the derivative of the above equation and we get $\nabla I^T \dot{x} + \frac{\partial I}{\partial t} = 0$. This is called the **optical flow constraint**. The quantity $\dot{x}$ is the *velocity* and the quantity we are interested in. But we cannot directly solve for $\dot{x}$ since we only have a projection of the velocity vector in the direction of the gradient $\nabla I$. So the Lucas-Kanade method makes that the additional assumption that there is *constant motion in a neighborhood* of any point. This means that for any $x'$ in a window $W(x)$ around $x$ we have $$\nabla I^T(x',t) \cdot \vec{v} + \frac{\partial I}{\partial t}(x',t) = 0$$ Since this condition doesn't hold in practice, we instead minimize $$E(v) = \int_{W(x)} \lvert \nabla I(x', t)^T v + I_t(x',t)\rvert^2 dx'$$ It is now more common to remove the square term and just minimize the absolute value for better results.

Now we can write $$\begin{align*}\lvert \nabla I(x', t)^T v + I_t(x',t)\rvert^2 &= \left(\nabla I('x,t)^T v\right)^2 + 2 I_t \nabla I(x',t)^Tv + \verb|const| \\
&= v^T \nabla I \nabla I^T v + 2I_t\nabla I^Tv  +\verb|const|\end{align*}$$ Thus, $$E(v) = v^T M v + 2q^T v + \verb|const|$$ where $$M = \int_{W(x)} \nabla I \nabla I^T dx' \text{     and     } q = \int_{W(x)} I_t \nabla I dx'$$ This means, $\frac{\partial E}{\partial v} = 0$ forces the condition $2Mv + 2q = 0$, and so $v = -M^{-1}q$ if $M$ is invertible.

It is possible for $M$ to not be invertible. Indeed, if the intensity is constant on the whole window, then we will have $M = 0$ and tracking optical flow in such a situation would not be possible. In the real world however, even constant color regions will have some texture and so $M$ will be invertible.

Another failure case is if the gradient is constant (in an edge, for example). In this case $M$ will just have rank 1. This amounts to the case where we are unable to estimate velocity cases along an edge.

Instead of the constant motion assumption we can assume that motion in a window is an *affine* function of the coordinate $x'$. In this case, the energy function is given by $$E(p) = \int_{W(x)} \lvert\nabla I(x')^TS(x')p + I_t(x')\rvert^2 dx'$$ This models more rich flows than simple translations. The analysis is the same as before, and we will have to invert a $6 \times 6$ matrix.

There are some complications in implementing the Lucas-Kanade algorithm. 

* Finding the vector $v$ depends on the small motion assumption that motion is on the range of 1 pixel. To handle this issue in higher resolution images, we reduce the resolution of images so that the motion is on the range of 1 pixel in the coarse scale. Better coarse refinement strategies improves performance drastically.
* The matrix $M$ (called the *structure tensor*) need not be invertible. We would however like both the eigenvalues be sufficiently non-zero so that the result is not conditioned. If the structure tensor is not invertible, we can estimate the *normal motion*, the component of the motion in the direction of the image gradient.

If $M(x)$ is invertible, we can construct the following simple feature tracking algorithm:

* For a given time step $t$ compute the structure tensor $$M(x) = \int_{W(x)} \begin{bmatrix} I_x^2  & I_xI_y \\ I_xI_y & I_y^2 \end{bmatrix} dx' $$ for each point $x$
* Mark all points $x$ for which the determinant is bigger than some threshold $\theta > 0$.
* For all these points, the local velocity is given by $$v(x,t) = -M(x)^{-1} \begin{bmatrix}\int I_x I_t dx' \\ I_y I_t dx' \end{bmatrix}$$

This is called the **KLT (Kanade, Lucas, Tomasi) tracker**. This is a very popular tracker because it is simple and fast. 

In the Harris corner detector you set up the structure tensor weighted by a Gaussian $$M = G_\sigma * \nabla I \nabla I^T = \int G_{\sigma}(x - x') \begin{pmatrix}I_x^2 & I_xI_y \\ I_xI_y & I_y^2 \end{pmatrix} (x') dx'$$  Then we pick those points for which $R = \det(M) + \kappa \text{Tr}(M)^2$ is sufficiently large. This selects many points which are corners. 

### **Wide Baseline Matching**

Corresponding points and regions may look very different in different views and determining correspondence is a challenge. Large points of the image plane will not match at all because they are not visible in the other image. Otherwise there may be specularities which leads to the image look very different depending on the position of the camera.

In the small baseline case we have the issue of *drift*, where small errors in the motion accumulate over time and the window gradually moves away from the point that was originally tracked. Also, if we try to extend the methods from the small baseline case to larger baselines, we have two considerations:

* Since the motion of the window between frames is no longer translational, one needs to generalize the motion model for the window $W(x)$.
* Since the illumination will change over time, one can replace the sum-of-squared-differences by the *normalized cross correlation*, which is more robust to illumination changes.

The **normalized cross correlation** is defined as:
$$
NCC(h)=\frac{\int_{W(x)}\left(I_{1}\left(x^{\prime}\right)-\bar{I}_{1}\right) \cdot \left(I_{2}\left(h\left(x^{\prime}\right)\right)-\bar{I}_{2}\right) d x^{\prime}}{\sqrt{\int_{W(x)}\left(I_{1}\left(x^{\prime}\right)-\bar{I}_{1}\right)^{2} d x^{\prime} \int_{W(x)}\left(I_{2}\left(h\left(x^{\prime}\right)\right)-\bar{I}_{2}\right)^{2} d x^{\prime}}}
$$ 

where $\bar{I}_1$ and $\overline{I}_2$ are the average intensities over the window $W(x)$. By subtracting this average intensity, the measure becomes invariant to additive intensity changes $I \to I+ \gamma$. Dividing by the intensity variances of each window makes the measure invariant to multiplication changes $I \to \gamma I$.

The normalized cross correlation is just the cosine of the angles between the vectors $I_1 - \overline{I}_1$ and $I_2 - \bar{I}_2$.

The normalized cross correlation allows handling brightness changes, but again, if the brightness is constant in the window, we get a $\frac{0}{0}$. So normalized cross correlation doesn't work in every situation.

NCC can be used to determine the optimal affine transformation between two given patches $h(x) = Ax + b$. We need to maximize the cross correlation with respect to the 2x2 matrix $A$ and the displacement $d$ and find $$\hat{A}, \hat{d} = \arg\max_{A,d} NCC(x \to Ax+b)$$

Efficiently finding appropriate optima however is a challenge.

### **Global methods for Optical Flow (CS565 PUCIT)**

Lucas-Kanade is a sparse method of determining optic flow as we calculate the flow only for some suitable points. Now we will look at global *dense* methods like Horn-Shunck's variational method.

At some given time $t$ the optical flow field is determined as minimizing the energy function $$ E(v) = \frac{1}{2} \sum_{x,y} \bigg(\underbrace{(\nabla I^T v + I_t)^2}_{\text{data term}} +  \underbrace{\alpha \cdot \lvert\lvert \nabla v \rvert\rvert_2^2}_{\text{smoothness term}}\bigg) $$ involving the function $v(x)$ representing the optical flow vector at pixel $x$. The smoothness term tries to favor solutions without much local variations in optical flow. At locations where no reliable flow estimation is possible (small $\lvert\lvert \nabla I \rvert\rvert$), the smoothness term will dominate over the data term.

Write $E(v) = \sum_{x,y} F(x,v, \nabla v)$. Then the Euler Lagrange Equations require that $\partial_x F_{\nabla v} - F_v = 0$. This gives a pair of equations for the two components of the optical flow vector $v$. We write $$F(x,y, u, v, u_x, u_y, v_x, v_y) = \left(I_x u + I_y v + I_t\right)^2 + \alpha (u_x^2 + u_y^2 + v_x^2 + v_y^2)$$ Then we have (upto a constant) $F_u = I_x(I_x u + I_y + I_t)$ and $F_{u_x} = \alpha u_x$ (all other terms can similarly be computed). The Euler-Lagrange equations then gives us the pair of equations $$\begin{align*}\alpha(u_{xx} + u_{yy}) - I_x(I_x u + I_y v + I_t) &= 0 \\ \alpha(v_{xx} + v_{yy}) - I_y(I_xu + I_yv + I_t) &= 0 \end{align*}$$ In any pixel we have $\nabla u = u_{xx} + u_{yy} = \frac{1}{h^2} \sum_{j \in  \mathcal{N}_i} (u_j - u_i)$ where $h$ is the pixel width (usually 1). Thus we obtain a pair of equations at each pixel. One of these equations for the $i$th pixel is $$\frac{\alpha}{h^2}\sum_{j \in \mathcal{N}_i} (u_j - u_i) - I_{xi} \left(I_{xi} u_i + I_{yi}v_i + I_{ti}\right) = 0$$ 

This can be solved by fixed point iteration $$\begin{align*}u_{i}^{(k+1)} &= \frac{\frac{\alpha}{h^2} \sum_{j \in \mathcal{N}_i} u_j^{(k)} - I_{xi} \left(I_{yi}v_i^{(k)} + I_{ti}\right)}{\frac{\alpha}{h^2} \lvert \mathcal{N}_i\rvert + I_{xi}^2}  \\ v_{i}^{(k+1)} &= \frac{\frac{\alpha}{h^2} \sum_{j \in \mathcal{N}_i} v_j^{(k)} - I_{yi} \left(I_{xi}u_i^{(k)} + I_{ti}\right)}{\frac{\alpha}{h^2} \lvert \mathcal{N}_i\rvert + I_{yi}^2} \end{align*}$$ where this process is repeated until convergence.

### **Graph Neural Networks and Applications (Youtube)**

The aim is to embed graphs into an embedding space so that they can be distinguished for a given task. 


More formally, a graph can either be given as a tuple $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ or by a square adjacency matrix. There may be additional node and edge features $x_v \in \mathbb{R}^F$ for $v \in \mathcal{V}$ and $e_{v,w} \in \mathbb{R}^d$ for all $(v,w) \in \mathcal{E}$.

Some problems in GNNs include 

* *node classification*, where we predict the class of a given node 
* *graph classification*, where we predict the class of an entire graph, 
* *link prediction*, where we predict the existence of an edge.

*Inductive learning* observes training data and learns from labelled training data. *Transductive learning* observes all data but only learns from labelled training data. This is also called semi-supervised learning.

Graph Neural networks can be used in 3D shape analysis, human pose estimation, visual scene understanding, autonomous driving and segmenting point clouds.

The nodes are embedded using an encoder and a similarity function (the dot product function) specifies how relations in the vector space map to relationships in the original network. 

The GNN training roadmap is to start with an input graph and encode it, pass it to the GNN to generate meaningful output node embeddings, and then apply task specific loss functions.

Graphs are represented as adjacency matrices and a GNN is an optimizable transformation on all attributes of the graph (nodes, edges, global-context) that preserves graph symmetries. This means that the matrix of the graph should be permutation invariant.

Using graphs for neural networks is difficult because:

* Graphs have arbitrary size and complex topological structure (so there is no spatial locality like grids)
* No fixed node ordering or reference point.
* Often dynamic and have multi-modal features.

Pre-Deep learning Graph machine learning had methods of embedding nodes and graphs. Node2Vec maps nearby nodes to similar embeddings and far away nodes to distinct embeddings. There are also graph embedding techniques (Graph kernels) that depend on hand-crafted features.

With the rise of deep learning we move towards end-to-end learning models which scale well with increase in data.

CNNs inject the inductive bias of translation invariance into the architecture.

In GNNs we want to learn local patterns of the graph. So we update node representations by repeatedly transforming and aggregating neighboring representations.

We can understand this with a message passing scheme. Each neighbor sends a message to a central node. $$m_{w,v}^{(\ell)} = \verb|Message|(h_{v}^{(\ell-1)}, h_w^{(\ell-1)})$$ These messages are aggregated across all neighbors (in a permutation-invariant way). $$a_{v}^{(\ell)} = \verb|Aggregate|\left(\{m_{w,v}^{\ell} : w \in \mathcal{N}(v)\}\right) $$ The output of the aggregation method is used to update the representation. $$h_{v}^{(\ell)} = \verb|Update|(h_{v}^{(\ell - 1)}, a_v^{(\ell)})$$



The message passing functions are trainable and differentiable and each node uses the same set of shared parameters. Message passing operations can be stacked.

Graph neural networks compute localized node embeddings $h_v^{(L)} \in \mathbb{R}^{\lvert \mathcal{V}\rvert \times h}$. We can use these embeddings in downstream tasks:

* In node classification, the node embeddings are passed through a fully connected network for classification.
* In graph classification, the sum of the node embeddings is passed through a FC network.
* In link prediction the appropriate pair of node embeddings is passed to a FC network.

The most popular implementation of the message passing scheme is $$h_{v}^{(\ell)} = \sigma \left(W_1 \underbrace{h_{v}^{(\ell-1)}}_{\text{skip connection}} + W_2 \sum_{w \in \mathcal{N}(v)} \underbrace{C_{v,w}}_{\text{normalization \\ coefficient}} h_{w}^{(\ell-1)}\right)$$ This scheme is flexible. If $C_{v,w} = 1$ then we have a static situation, but it can also be data-dependent. A node can attend to the neighboring nodes that have valuable information.

## **Chapter 5: Reconstruction from Two Views**

We want to reconstruct 3D geometry of cameras and points given a set of corresponding points in two frames taken with the same camera. We will assume the scene is static and that the intrinsic camera parameters are known (calibration). The goal is to estimate the camera motion and the 3D locations of points. 

The projections of a point $X$ onto the two images are denoted $x_1$ and $x_2$. The optical centers of each image are denoted by $o_1$ and $o_2$. The intersections of the line $(o_1,o_2)$ with each image plane are called *epipoles* $e_1$ and $e_2$ and the intersection between the *epipolar plane* $(o_1, o_2, X)$ and the image planes are called the *epipolar lines* $\ell_1$ and $\ell_2$. 

*Bundle adjustment* is an approach to solving the 3D reconstruction problem where we optimize the least square error of a choice of camera motion matrix and 3D world coordinates. This is a non-convex optimization problem in high dimensions, which makes it challenging. 

### **The Epipolar Constraint and Essential Matrices**



We suppose that $K = 1$ since we know the intrinsic camera parameters.  So we have $\lambda_1 x_1 = X$ and $\lambda_2 x_2 = RX + T$, where $\lambda_1$ and $\lambda_2$ are unknown depth parameters. Thus, $\lambda_2 x_2 = \lambda_1 Rx_1 + T$. We wish to eliminate $T$, and we can do this with the cross product matrix $\hat{T}$. So we have $$\lambda_2 \hat{T} x_2 = \lambda_1 \hat{T}Rx_1$$ But $\hat{T}x_2$ is orthogonal to $x_2$, so we get the *epipolar constraint* $$x_2^T \hat{T}R x_1 = 0$$

This constraint provides a relation between the 2D point coordinates of a 3D point in two images and the camera transformation parameters. The matrix $E = \hat{T}R$ is called the **essential matrix** and the space of essential matrix is called the essential space $$\mathcal{E} = \left\{\hat{T} R \mid R \in SO(3), T \in \mathbb{R}^3 \right\}$$ Huang and Faugeras characterized essential matrices as those which have an SVD $E = U\Sigma V^T$ where $\Sigma = \verb|diag|(\sigma, \sigma, 0)$ for some $\sigma > 0$ and $U,V \in SO(3)$. In fact, 

**Theorem** (*Pose recovery from the essential matrix*) 
There exist exactly two relative poses $(R,T)$ corresponding to an essential matrix $E \in \mathcal{E}$. If $E = U \Sigma V^T$, we have $$\begin{align*} ((\hat{T}_1, R_1) &= \Big(UR_{z}\left(\tfrac{\pi}{2}\right)\Sigma U^T, UR_z^T\left(\tfrac{\pi}{2}\right)V^T\Big) \\ ((\hat{T}_2, R_2) &= \Big(UR_{z}\left(-\tfrac{\pi}{2}\right)\Sigma U^T, UR_z^T\left(-\tfrac{\pi}{2}\right)V^T\Big)\end{align*}$$ In general, only one of these two gives meaningful (positive) depth.

We would like to recover the essential matrix $E$ from point correspondences, but in general, the recovered matrix will not satisfy the required constraints. We can resolve this problem in two ways:

* Project the recovered matrix $E$ onto the essential space.
* Optimize the epipolar constraints in the essential space.

The second approach is in principle more accurate, but it involves a nonlinear constrained optimization. We will pursue the first approach.

### **The Eight Point Algorithm**




First we rewrite the epipolar constraint as a scalar product in the elements of the matrix $E$ and the coordinates of $x_1$ and $x_2$. Let $E  = \begin{bmatrix} e_{11} & e_{21} & \ldots & e_{33}\end{bmatrix}^T \in \mathbb{R}^{9}$ and let $a = x_1 \otimes x_2$ be the Kronecker product of the two coordinates. This means $$a = \begin{bmatrix} x_1x_2 & x_1y_2 & x_1 z_2 & y_1z_1 &  y_1y_2 & y_1z_2 & z_1x_2 & z_1y_2 & z_1z_2\end{bmatrix}^T$$ Then $$x_2^T E x_1 = a^T E_s = 0$$ This is just a linear constraint, so different point correspondences will give further equations that will constrain the system. However, we will not be able to eliminate the scale freedom. Further information from the scene is required to specify the scale (for example known sizes of objects in the real world).

In certain degenerate cases we will not be able to identify a 1 dimensional space of solutions even if we have 8 or more points. For example, if all the points lie on a line or on a plane. There are more degeneracies involving quadrics, but that doesn't come up often in practice.

Even if we normalize $E$, we will not be able to decide the sign of $E$. So we will have 4 possible estimates for the rotation and translation effects. It is usually easy in practice to decide the correct one.

For a matrix $F = U \verb|diag|(\lambda_1,\lambda_2,\lambda_3) V^T$ with $\lambda_1 \geq \lambda_2 \geq\lambda_3$ the essential matrix $E$ which minimizes the frobenius norm $\lvert\lvert E - F \rvert\rvert_2^2$ is given by $$E = U\verb|diag|(\sigma, \sigma, 0)V^T$$ where $\sigma = \frac{\lambda_1 + \lambda_2}{2}$. Since we are unable to estimate the scale factor, we set $\sigma = 1$ in practice.

The eight point image fails if the translation is exactly $0$ since then $E = 0$ and nothing can be recovered. In the case of infinitesimal view point change, one can adapt the eight point algorithm to the continuous motion case where the the epipolar constraint is replaced by the continuous epipolar constraint.Rather than recovering $(R,T)$ we recover the linear and angular velocity of the camera.

### **Structure Reconstruction**

We recover the essential matrix $E$ but only up to an arbitrary scale factor. This means that we are unable to identify the distance between the camera centers $\gamma$. So we have $$\lambda_{2}^j x_2^j = \lambda_1^j Rx_1^j + \gamma T$$ for each point $j=1,\ldots, n$.



We can eliminate $x_2^j$ using the cross product and we get the system $$\begin{pmatrix}\widehat{x_2^{j}}Rx_1^j& \widehat{x_2^{j}}T \end{pmatrix}\begin{pmatrix} \lambda_1^j \\ \gamma\end{pmatrix} = 0$$ We can combine this system to a bigger matrix $$M = \begin{pmatrix} \widehat{x_2^1} R x_1^1 & 0 & \ldots & \widehat{x_2^1} T  \\ 0 & \widehat{x_2^2} R x_1^2 & \ldots & \widehat{x_2^2} T \\
\vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \ldots & \widehat{x_2^n} T \end{pmatrix}$$ and so the system becomes $M\lambda = 0$. We may not be able to directly solve this system in practice, so we will choose to minimize $\lvert M\lambda\rvert^2$ subject to $\lvert\lambda\rvert=1$. This will correspond to an eigenvector of $M^TM$ of the smallest eigenvalue. Note that we only obtain depth estimation upto a global scale. 

If the correspondences are inaccurate or incorrect there is no guarantee that the estimated $R$ and $T$ will be close to the true solution. One solution is to use bundle adjustment and minimize $$\phi(x_1, R, T, \lambda) = \sum_{j}\lvert\lvert  \tilde{x_1}^j - x_1^j \rvert\rvert^2 + \lvert\lvert \tilde{x_2}^j - \pi\left(R \lambda_1^j x_1^j + T\right)\rvert\rvert^2$$ where $\tilde{x_i}^j$ are the measured coordinates  and $x$ are the true coordinates


### **Degenerate Configurations**

The eight point algorithm only provides unique solutions if all 3D points rae in general position. This is no longer the case for certain degenerate configurations for which all the points lie on certain 2d surfaces called *critical surfaces*. Typically, these critical surfaces are described by a quadratic equation in the there point coordinates.

For the structure-from-motion problem in the context of points on a plane, one can exploit additional constraints which lead to the so called *Four point algorithm*.

Let us assume  that all the points lie on the plane $\frac{1}{d}N \cdot \mathbf{X} = 1$. Then $$X_2 = RX_1 + T = \left(R + \frac{1}{d}TN^T\right) X_1 = HX_1$$ where $H$ is the **homography** $3 \times 3$ matrix. Inserting 2D coordinates, we get $\lambda_2 x_2 = H \lambda     _1 x_1$ so that $x_2 \sim Hx_1$. This expression is called a *planar homography*. This homography matrix depends on the transformation and plane parameters but not on the 3D or 2D coordinates.

Again, multiplying by $\widehat{x_2}$ we get $$\widehat{x_2} H x_1 = 0$$ This is called the *planar epipolar constraint* or the *planar homography constraint*. Again, we can cast this equation into the form $a^TH$ where $a = x_1 \otimes \widehat{x_2} \in \mathbb{R}^{9 \times 3}$. This matrix has rank 2 and so we need only 4 points, leading to the name "4-point algorithm".

### **The Case of an Uncalibrated Camera**

In general, the transformation between 3D coordinates and 2D coordinates is $$\lambda x' = K\Pi_0 g X = (KR, KT)X$$ with the intrinsic parameter matrix $$K = \begin{pmatrix} fs_x & s_\theta & o_x \\ 0  & fs_y & o_y \\ 0 & 0 & 1 \end{pmatrix}$$ Assume we don't have the camera matrix $K$ before hand. We will get the epipolar constraint $$x_2^{'T} K^{-T} \hat{T} R K^{-1} x_1' = 0$$ So $$x_2'F x_1' = 0$$ where $F = K^{-T}\hat{T}R K^{-1} = K^{-T}EK$ is called the *fundamental matrix*.

It is straightforward to extend the eight-point algorithm to this case, but it is less straightforward to see how to proceed from there. One cannot impose a Strong constraint on the specific structure of the fundamental matrix. Also, for a given fundamental matrix $F$, there doesn't exist a finite number of decompositions into extrinsic parameters $R,T$ and intrinsic parameters $K$. So one can only determine projective reconstruction's (up to a projective transformation). As a solution, one typically chooses a canonical representation from the family of possible reconstructions.

## **Chapter 6: Reconstruction from Multiple Views**



### **Preimages and Coimages from Multiple Views**

A preimage of multiple images of a point or a line is the set of 3D points that gives rise to that set of images.  

For a moving camera at time $t$ let $x(t)$ denote the image coordinates of a 3D point $X$ in homogeneous coordinates: $$\lambda(t) x(t) = K(t)\Pi_0 g(t)X$$ where $\lambda(t)$ is the depth of the point, $K(t)$ denotes the intrinsic parameters, $\Pi_0$ the generic projection, and $$g(t) = \begin{pmatrix} R(t) & T(t) \\ 0 & 1\end{pmatrix} \in SE(3)$$ denotes the rigid body motion at time $t$.

Let us consider a 3D line $L = \left\{X \mid X = X_0 + \mu V, \mu \in \mathbb{R}\right\}$ where $X_0 = [X_0,Y_0,Z_0, 1]^T$ in homogeneous coordinates and $V = [V_1,V_2, V_3, 0]^T$. The preimage of the image of $L$ at time $t$ is a plane with a normal $\ell(t)$. We must have $$ \ell(t)^T x(t) = \ell(t)^T K(t) \Pi_0 g(t)X = 0$$ Suppose we are given $m$ images at times $t_1,\ldots, t_m$ where $\lambda_i = \lambda(t_i)$, $x_i = x(t_i)$, $\ell_i = \ell(t_i)$ an  $\Pi_i = K(t_i) \Pi_0 g(t_i)$. With this notation, we can relate the $i$ th image of a point $p$ its world coordinates $X$: $$\lambda_i x_i = \Pi_i X$$ and the $i$ th co-image of a  line L to its world coordinates $(X_0, V)$ $$\ell_i^T \Pi_i X_0 = \ell_i^T \Pi_i V = 0$$

The above equations contain the 3D parameters of points and lines as unknowns. As in the two-view case, we wish to eliminate these unknowns so as to obtain relationships between the 2D projections and the camera parameters. In the two-view case, an elimination of the 3D coordinates leads to the *epipolar constraint* for the essential matrix $E$ or fundamental matrix $F$ depending on whether we are in the calibrated or uncalibrated case. The 3D coordinates (depth values $\lambda_i$) could subsequently be obtained from another constraint. 

There exist different ways to eliminate the 3D parameters leading to different kinds of constraints which have been studied in Computer Vision.

### **From Preimages to Rank Constraints**

Consider images of a 3D point $X$ seen in multiple views. Then we have the matrix equation $\mathcal{I}\vec{\lambda} = \Pi X$ where $$\mathcal{I}\vec{\lambda} = \begin{bmatrix} \mathbf{x}_1 & 0 & \ldots & 0 \\ 0 & \mathbf{x}_2 & \ldots & 0 \\  \vdots & \vdots & \ddots  & \vdots  \\ 0 & 0 & \ldots &\mathbf{x}_m\end{bmatrix} \begin{bmatrix}\lambda_1 \\ \lambda_2 \\ \vdots \\ \lambda_m \end{bmatrix} = \begin{bmatrix} \Pi_1 \\ \Pi_2 \\ \vdots \\ \Pi_m\end{bmatrix} X = \Pi X$$ Here $\mathcal{I}$ is the *image matrix*, $\vec{\lambda}$ is the depth scale vector, and $\Pi$ is the *multiple view projection matrix*.

Write $$ N_p = \left(\Pi, \mathcal{I}\right) = \begin{bmatrix} \Pi_1 & \mathbf{x}_1 & 0 & \ldots & 0 \\ \Pi_2&  0 & \mathbf{x}_2 & \ldots & 0 \\  \vdots & \vdots & \vdots & \ddots  & \vdots  \\ \Pi_m & 0 & 0 & \ldots &\mathbf{x}_m\end{bmatrix} \in \mathbb{R}^{3m \times (m+4)}$$ The columns of $N_p$ should be linearly dependent, so we must have the *rank constraint* $$\verb|rank|(N_p) \leq m + 3$$ In fact, $u = (X, \vec{\lambda})^T$ is in the null space. We will later want to require that the rank is exactly $m+3$. 

Now would like to decouple structure and motion just as in the two-view case. We can apply similar transformations in this case. Writing  $$ \mathcal{I}^\perp = \begin{bmatrix} \widehat{\mathbf{x}}_1 & 0 & \ldots & 0 \\ 0 &  \widehat{\mathbf{x}}_2 & \ldots & 0 \\  \vdots & \vdots & \ddots  & \vdots  \\ 0 & 0 & \ldots & \widehat{\mathbf{x}}_m\end{bmatrix} \in \mathbb{R}^{3m \times 3m}$$ we have $\mathcal{I}^\perp \mathcal{I} = 0$. So we get $\mathcal{I}^\perp \Pi X = 0$. This means that $X$ is in the null space of the matrix $$W_p = \mathcal{I}^\perp \Pi = \begin{bmatrix} \widehat{\mathbf{x}}_1 \Pi_1 \\ \vdots \\ \widehat{\mathbf{x}}_m \Pi_m \end{bmatrix} \in \mathbb{R}^{3m \times 4}$$ Thus we have $\verb|rank|(W_p) = \verb|rank|(N_p) - m \leq 3$

We can derive a similar rank constraint for lines. If we have only two images of a line, then we will not be able to derive constraints on the camera transformation because any two images of a line will always intersect in a line. But if we have multiple images, we can consider those camera motions for which the intersection of all the preimages is a line (rather than a point). 

We have $$\ell_i^T \Pi_i X_0 = \ell_i^T \Pi_i V = 0$$ This means that for $$W_\ell = \begin{bmatrix} \ell_1^T \Pi_1 \\ \vdots \\ \ell_1^T \Pi_m \end{bmatrix} \in \mathbb{R}^{m \times 4}$$ we must have $\verb|rank|(W_\ell) \leq 2 $. This is a non-trivial constraint for $m > 2$.

### **The Multiple View Matrix**

We will now write these constraints in a more compact manner.

 Let us assume that we have $m$ images, the first of which is in world coordinates. Then we have projection matrices $\Pi_1 = [I,0], \Pi_2 = [R_2, T_2], \ldots, \Pi_m = [R_m, T_m] \in \mathbb{R}^{3 \times 4}$. In general for uncalibrated cameras with $K_i \neq I$, $R_i$ will not be an orthogonal rotation matrix but rather an arbitrary invertible matrix.  Again we define the matrix $W_p$ as before, but now we multiply it with $D_p$, a full rank $4\times 5$ matrix. $$ W_pD_p = \begin{bmatrix} \widehat{\mathbf{x}}_1\Pi_1 \\ \widehat{\mathbf{x}}_2\Pi_2 \\ \vdots \\ \widehat{\mathbf{x}}_m \Pi_m\end{bmatrix}\begin{bmatrix}\widehat{\mathbf{x}}_1 & \mathbf{x}_1 & 0 \\ 0 & 0 & 1 \end{bmatrix} = \begin{bmatrix}\widehat{\mathbf{x}}_1 \widehat{\mathbf{x}}_1 & 0 & 0 \\ \widehat{\mathbf{x}}_2R_2\widehat{\mathbf{x}}_1 & \widehat{\mathbf{x}}_2 R_2 \mathbf{x}_1 & \widehat{\mathbf{x}}_2 T_2 \\ \vdots & \vdots & \vdots \\ \widehat{\mathbf{x}}_m R_m \widehat{\mathbf{x}}_m & \widehat{\mathbf{x}}_m R_m \mathbf{x}_1 & \widehat{\mathbf{x}}_m T_m \end{bmatrix} \in \mathbb{R}^{3m \times 5}$$ We pick out the following submatrix from $W_pD_p$ and denote it $M_p$, $$ M_p = \begin{bmatrix}\widehat{\mathbf{x}}_2R_2 \mathbf{x_1} & \widehat{\mathbf{x}}_2 T_2 \\ \vdots & \vdots \\ \widehat{\mathbf{x}}_m R_m \mathbf{x}_m & \widehat{\mathbf{x}}_m R_m \end{bmatrix} \in \mathbb{R}^{3(m-1) \times 2}$$ Then the condition $\verb|rank|(W_p) \leq 3$ becomes the constraint $\verb|rank|(M_p) \leq 1$ because $\hat{x_1}\hat{x_1}$ has rank 2. Indeed $$\begin{bmatrix} 0 & x & y \\ -x &  0 & z \\ -y  & -z & 0 \\ \end{bmatrix}\begin{bmatrix}0 & x & y \\ -x & 0 & z \\ -y & -z & 0 \end{bmatrix} = \begin{bmatrix}-x^2 -y^2 & -yz & xz\\-yz & -x^2 - z^2 &  -xy\\ xz & -xy  & -y^2 - z^2 \end{bmatrix}$$ has rank 2 since the determinant of the top-left $2\times 2$ sub-matrix is $(x^2 + y^2)(x^2+z^2) - y^2z^2 = x^4 + x^2z^2 + y^2x^2 > 0$ if $x > 0$ (which we may assume by symmetry).

This matrix $M_p$ is called the **multiple view matrix**. We will show that the multiple view matrix contains epipolar constraints for each image with the first. 

We have $\widehat{x}_i (\alpha R_i x_1 + \beta T_i) = 0$ for some scalars $\alpha, \beta$. We assume $\beta \neq 0$ (there are no pure rotations), so we may assume $\beta = 1$. Then  $$x_i \times T_i = \alpha (x_i \times R_i x_1)$$ Since $x_i \times T_i$ is proportional to the normal on the plane spanned by $x_i$ and $T_i$, and $x_i \times R_ix_1$ is proportional to the normal to the plane spanned by $x_i$ and $R_i x_1$, the linear dependence is equivalent to saying that the three vectors $x_i, T$, and $R_i x_1$ are coplanar. Thus, $x_i^T \hat{T_i}(R_i x_1) = 0$.



For any non-zero vectors $a_i, b_i \in \mathbb{R}^3$, the matrix $$\begin{bmatrix}a_1 & b_1 \\ a_2 & b_2 \\ \vdots&\vdots \\ a_n & b_n \end{bmatrix} \in \mathbb{R}^{3n \times 2}$$ is rank deficient if and only if $a_ib_j^T = a_jb_i^T$ for all $i,j$. We apply this to the multiple view matrix $M_p$ and we get $$\widehat{\mathbf{x}}_i R_i \mathbf{x}_1 \left(\widehat{\mathbf{x}}_j T_j\right)^T = \widehat{\mathbf{x}}_i T_i \left(\widehat{\mathbf{x}}_j R_j \mathbf{x}_1\right)^T$$ This gives the *trilinear constraint* $$\widehat{\mathbf{x}}_i\left(T_i \mathbf{x}_1^T R_j^T - R_i \mathbf{x}_1 T_j^T\right)\widehat{\mathbf{x}}_j = 0$$ This is a matrix equation giving $9$ scalar trilinear equations of which four are linearly independent. These